In [4]:
# =====================================================
# DEMO EN VIVO: Reconocimiento facial (detección → embedding → SVM)
# =====================================================
# Requiere:
# - OpenCV (cv2)
# - numpy
# - scikit-learn
# - Tus artefactos entrenados:
#   output/recognizer_svm.pickle
#   output/label_encoder.pickle
#   output/threshold.pkl (opcional)
#   output/open_set_meta.pkl (opcional, con centroides por clase)
# - Modelos:
#   models/deploy.prototxt
#   models/res10_300x300_ssd_iter_140000.caffemodel
#   models/openface_nn4.small2.v1.t7
# =====================================================

import cv2
import numpy as np
import pickle
import time
from pathlib import Path
from sklearn.metrics.pairwise import cosine_distances

# -----------------------------------------------------
# 1️⃣ Cargar clasificador (SVM), codificador y umbral
# -----------------------------------------------------
clf = pickle.load(open("output/recognizer_svm.pickle", "rb"))
le  = pickle.load(open("output/label_encoder.pickle", "rb"))

THRESH_PATH = Path("output/threshold.pkl")
if THRESH_PATH.exists():
    BEST_T = float(pickle.load(open(THRESH_PATH, "rb")))
else:
    BEST_T = 0.7  # valor por defecto
    print(f"ℹ️ No se encontró {THRESH_PATH.name}. Usando umbral por defecto: {BEST_T}")

# -----------------------------------------------------
# 2️⃣ Cargar modelo de detección facial (SSD OpenCV)
# -----------------------------------------------------
FACE_PROTO = Path("models/deploy.prototxt")
FACE_MODEL = Path("models/res10_300x300_ssd_iter_140000.caffemodel")
detector = cv2.dnn.readNetFromCaffe(str(FACE_PROTO), str(FACE_MODEL))

def detectar_caras_bgr(img, conf_thresh=0.5):
    """Detecta rostros en una imagen BGR y devuelve [(x1,y1,x2,y2,score), ...]"""
    (h, w) = img.shape[:2]
    blob = cv2.dnn.blobFromImage(
        cv2.resize(img, (300, 300)),
        1.0,
        (300, 300),
        (104.0, 177.0, 123.0)
    )
    detector.setInput(blob)
    detections = detector.forward()
    boxes = []

    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence < conf_thresh:
            continue
        box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
        (x1, y1, x2, y2) = box.astype("int")
        boxes.append((x1, y1, x2, y2, confidence))

    return boxes

# -----------------------------------------------------
# 3️⃣ Cargar modelo de embeddings (OpenFace)
# -----------------------------------------------------
EMBED_MODEL = Path("models/openface_nn4.small2.v1.t7")
embedder = cv2.dnn.readNetFromTorch(str(EMBED_MODEL))

def embedding_cara(img, box):
    """Obtiene el embedding (vector 128D) del rostro recortado."""
    (x1, y1, x2, y2, *_) = box
    face = img[y1:y2, x1:x2]
    if face.size == 0:
        return None
    face_blob = cv2.dnn.blobFromImage(
        cv2.resize(face, (96, 96)),
        1.0/255,
        (96, 96),
        (0, 0, 0),
        swapRB=True,
        crop=False
    )
    embedder.setInput(face_blob)
    vec = embedder.forward()
    return vec.flatten()

# -----------------------------------------------------
# 3.1️⃣ Cargar meta de open set (centroides por clase) de forma segura
# -----------------------------------------------------
OPEN_SET_META_PATH = Path("output/open_set_meta.pkl")

class_means = None
class_thr   = None

if OPEN_SET_META_PATH.exists():
    open_set_meta = pickle.load(open(OPEN_SET_META_PATH, "rb"))

    # Puede que sea un float (como en tu error) o un dict con info
    if isinstance(open_set_meta, dict) and \
       "class_means" in open_set_meta and \
       "class_thr"   in open_set_meta:

        class_means = open_set_meta["class_means"]
        class_thr   = open_set_meta["class_thr"]
        print("✅ open_set_meta.pkl cargado correctamente con centroides por clase.")
    else:
        print("⚠️ open_set_meta.pkl NO contiene 'class_means' y 'class_thr'.")
        print("   Se asumirá que solo tienes un umbral global y se ignorará open set por centroides.")
else:
    print("ℹ️ No se encontró open_set_meta.pkl, solo se usará el umbral de probabilidad.")

# -----------------------------------------------------
# 4️⃣ Función principal: demo webcam
# -----------------------------------------------------
def demo_webcam(cam_index=0, conf_det=0.5, conf_cls=None,
                show_fps=True, save_dir="snapshots", resize_ratio=None):
    """
    Reconocimiento en vivo con cámara:
      [s] guarda snapshot
      [q] o [ESC] sale
    """
    thr = float(conf_cls) if conf_cls is not None else BEST_T
    Path(save_dir).mkdir(parents=True, exist_ok=True)

    cap = cv2.VideoCapture(cam_index)
    if not cap.isOpened():
        raise RuntimeError(f"No se pudo abrir la cámara index={cam_index}")

    def draw_label(img, text, x, y, color):
        """Texto con fondo para mejor visibilidad."""
        (tw, th), _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
        cv2.rectangle(img, (x, y - th - 8), (x + tw, y), (0, 0, 0), -1)
        cv2.putText(img, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    prev = time.time()
    fps = 0.0
    print("▶ Demo iniciada. Presiona [q]/[ESC] para salir, [s] para snapshot.")

    try:
        while True:
            ok, frame = cap.read()
            if not ok:
                print("⚠️ No se pudo leer frame de la webcam.")
                break

            if resize_ratio and 0 < resize_ratio < 1.0:
                frame = cv2.resize(frame, None, fx=resize_ratio, fy=resize_ratio)

            boxes = detectar_caras_bgr(frame, conf_thresh=conf_det)

            for box in boxes:
                (x1, y1, x2, y2, *rest) = box
                h, w = frame.shape[:2]
                x1, y1 = max(0, x1), max(0, y1)
                x2, y2 = min(w - 1, x2), min(h - 1, y2)

                vec = embedding_cara(frame, (x1, y1, x2, y2, *rest))
                if vec is None:
                    continue

                # 1) Predicción SVM
                probs = clf.predict_proba([vec])[0]
                order = np.argsort(probs)[::-1]
                j1, j2 = order[0], order[1]
                p1, p2 = float(probs[j1]), float(probs[j2])
                pred_name = le.classes_[j1]

                # 2) Normalizar embedding (si el clf es un Pipeline con 'normalizer')
                try:
                    normalizer = clf.named_steps["normalizer"]
                    vec_norm = normalizer.transform([vec])[0]
                except Exception:
                    # Si no existe 'normalizer', usamos el vector tal cual
                    vec_norm = vec

                is_unknown = False

                # --- Regla por probabilidad + gap ---
                thr_abs = thr      # del threshold.pkl o por parámetro
                thr_gap = 0.15

                if (p1 < thr_abs) or (p1 - p2 < thr_gap):
                    is_unknown = True

                # --- Regla por distancia al centroide (si hay meta válida) ---
                if (class_means is not None) and (pred_name in class_means):
                    mu = class_means[pred_name]
                    d = float(cosine_distances([vec_norm], [mu])[0, 0])

                    # umbral por clase
                    d_thr = class_thr[pred_name]
                    if d > d_thr:
                        is_unknown = True

                if is_unknown:
                    name = "Desconocido"
                    color = (0, 0, 255)
                else:
                    name = pred_name
                    color = (0, 255, 0)

                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                draw_label(frame, f"{name}: {p1:.2f}", x1, y1, color)

            # Mostrar FPS
            if show_fps:
                now = time.time()
                dt = now - prev
                if dt > 0:
                    fps = 0.9 * fps + 0.1 * (1.0 / dt) if fps > 0 else 1.0 / dt
                prev = now
                draw_label(frame, f"FPS: {fps:.1f}", 10, 25, (0, 255, 255))

            # Mostrar en ventana
            cv2.imshow("Reconocimiento en vivo", frame)
            key = cv2.waitKey(1) & 0xFF

            # Guardar snapshot
            if key == ord('s'):
                snap_path = Path(save_dir) / f"snapshot_{int(time.time())}.jpg"
                cv2.imwrite(str(snap_path), frame)
                print(f"💾 Snapshot guardado: {snap_path}")

            # Salir
            if key == ord('q') or key == 27:  # 27 = ESC
                break

    except KeyboardInterrupt:
        print("\n⛔ Interrumpido por el usuario.")
    finally:
        cap.release()
        cv2.destroyAllWindows()
        print("⏹ Demo finalizada.")

# -----------------------------------------------------
# 5️⃣ Ejecutar demo
# -----------------------------------------------------
if __name__ == "__main__":
    demo_webcam(
        cam_index=0,
        conf_det=0.5,
        conf_cls=None,   # si quieres forzar otro umbral de probabilidad, pon un float aquí
        resize_ratio=0.8
    )


⚠️ open_set_meta.pkl NO contiene 'class_means' y 'class_thr'.
   Se asumirá que solo tienes un umbral global y se ignorará open set por centroides.
▶ Demo iniciada. Presiona [q]/[ESC] para salir, [s] para snapshot.
⏹ Demo finalizada.
